<a href="https://colab.research.google.com/github/sujith2303/ML_From_Scratch/blob/main/Transformers_from_Scratch_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.nn as nn
import torch

In [ ]:
class MLP(nn.Module):
  def __init__(self,input_dim,output_dim,rank):
    super().__init__()
    self.layer = nn.Linear(input_dim,output_dim)
    self.lora_A = nn.Parameter(torch.zeros(rank,output_dim))
    self.lora_B = nn.Parameter(torch.zeros(input_dim,rank))
    nn.init.normal_(self.lora_A)
    nn.init.normal_(self.lora_B)
  def forward(self,x):
    return self.layer(x) + x @ self.lora_B @ self.lora_A


In [ ]:
a = MLP(10,5,2)

In [ ]:
a.layer.weight.requires_grad = False
a.layer.bias.requires_grad = False

In [ ]:
for name, param in a.named_parameters():
  print(name,param,param.requires_grad)

lora_A Parameter containing:
tensor([[ 0.0704,  0.6876,  2.6952,  0.4876, -1.1048],
        [ 0.8194,  0.8655,  0.4910,  0.5457,  0.7770]], requires_grad=True) True
lora_B Parameter containing:
tensor([[-0.3533, -0.2863],
        [ 0.8812, -0.7112],
        [ 1.3064,  2.4411],
        [-1.3770, -0.3194],
        [-0.2807, -1.1665],
        [ 0.3467, -0.1804],
        [ 0.2115, -1.2617],
        [ 1.8283, -0.0567],
        [ 0.9864,  1.2137],
        [ 1.2741,  0.9932]], requires_grad=True) True
layer.weight Parameter containing:
tensor([[ 0.0696,  0.0941, -0.2601,  0.2813,  0.2521,  0.0981,  0.0656, -0.2960,
         -0.0392,  0.0362],
        [ 0.3032, -0.2225,  0.0082, -0.0442, -0.2522,  0.1042,  0.1358,  0.1444,
          0.0823, -0.2012],
        [ 0.1838, -0.2032, -0.1282, -0.0174, -0.1534,  0.2721, -0.1783,  0.1236,
         -0.2177,  0.0097],
        [-0.2884, -0.1502,  0.0343,  0.2799, -0.3026, -0.0723,  0.0142,  0.0437,
         -0.0480,  0.0276],
        [ 0.2473,  0.1981,  0

In [ ]:
a(torch.randn(10))

tensor([ 0.4993, -0.0523, -4.6377, -0.5134,  3.4244], grad_fn=<AddBackward0>)

In [ ]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(a.parameters(),lr=5e-5,)

In [ ]:
X = torch.ones(1,10)
y = torch.ones(1,5)*35

In [ ]:
X,y

(tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 tensor([[35., 35., 35., 35., 35.]]))

In [ ]:
for epoch in range(2000):
  output = a(X)
  l = loss(output,y)
  l.backward()
  optimizer.step()
  print(l)

In [ ]:
a(torch.ones(10))

tensor([35.4564, 33.4413, 33.5289, 32.9141, 36.6577], grad_fn=<AddBackward0>)

In [ ]:
total = 0
some = 0
for i,j in a.named_parameters():
  if j.requires_grad:
    some+=j.numel()
  total+=j.numel()
print("Trainable parameters:-",some/total)

Trainable parameters:- 0.35294117647058826


In [ ]:
some,total

(0, 5)

In [ ]:
optimizer.state

defaultdict(dict,
            {Parameter containing:
             tensor([[ 2.0505, -0.1295, -0.4679,  1.9876,  3.3249],
                     [ 0.3922,  4.5915,  5.3520,  0.3100, -2.1798]], requires_grad=True): {'step': tensor(13016.),
              'exp_avg': tensor([[ 46308.0898,  -3028.9744, -25398.5508, -34328.2109, -54617.7969],
                      [ 25982.0488,  -4197.7524, -31718.5371, -17310.4453, -13126.2461]]),
              'exp_avg_sq': tensor([[1.7317e+09, 9.7815e+06, 4.8879e+08, 5.8698e+08, 2.5055e+09],
                      [5.4433e+08, 1.3326e+07, 8.8502e+08, 1.5559e+08, 1.4926e+08]])},
             Parameter containing:
             tensor([[ 0.7369,  0.4212],
                     [ 1.9712, -0.0037],
                     [ 2.3963,  3.1482],
                     [-0.2866,  0.3880],
                     [ 0.8094, -0.4590],
                     [ 1.4367,  0.5270],
                     [ 1.3016, -0.5541],
                     [ 2.9181,  0.6507],
                     [ 2.

# 1.58 bit LLMs

# Consider only unmasked data for loss


In [ ]:
import torch.nn as nn
import torch

In [ ]:
max_length = 4
embedding_dim = 10
pos = torch.arange(max_length)
i = torch.arange(0,embedding_dim//2,out = torch.empty((embedding_dim//2,1)))
pos_embeddings = torch.empty(1,embedding_dim,max_length)
pos_embeddings[:,0::2,:] = torch.sin(pos/torch.pow(10000,(2*i/embedding_dim)))
pos_embeddings[:,1::2,:] = torch.cos(pos/torch.pow(10000,(2*i/embedding_dim)))
pos_embeddings

In [ ]:
# prompt: generate positional embeddings of transformers using pytorch inbuild

import torch

max_length = 4
embedding_dim = 10

# Create position indices
pos = torch.arange(max_length)

# Create embedding dimension indices
i = torch.arange(0, embedding_dim // 2, out=torch.empty((embedding_dim // 2, 1)))

# Initialize the positional embeddings tensor
pos_embeddings = torch.empty(1, max_length, embedding_dim)

# Calculate sine and cosine embeddings
pos_embeddings[:, :, 0::2] = torch.sin(pos / torch.pow(10000, (2 * i / embedding_dim)))
pos_embeddings[:, :, 1::2] = torch.cos(pos / torch.pow(10000, (2 * i / embedding_dim)))

pos_embeddings

tensor([[[ 0.0000e+00,  8.4147e-01,  9.0930e-01,  1.4112e-01],
         [ 1.0000e+00,  5.4030e-01, -4.1615e-01, -9.8999e-01],
         [ 0.0000e+00,  1.5783e-01,  3.1170e-01,  4.5775e-01],
         [ 1.0000e+00,  9.8747e-01,  9.5018e-01,  8.8908e-01],
         [ 0.0000e+00,  2.5116e-02,  5.0217e-02,  7.5285e-02],
         [ 1.0000e+00,  9.9968e-01,  9.9874e-01,  9.9716e-01],
         [ 0.0000e+00,  3.9811e-03,  7.9621e-03,  1.1943e-02],
         [ 1.0000e+00,  9.9999e-01,  9.9997e-01,  9.9993e-01],
         [ 0.0000e+00,  6.3096e-04,  1.2619e-03,  1.8929e-03],
         [ 1.0000e+00,  1.0000e+00,  1.0000e+00,  1.0000e+00]]])

In [ ]:
transformer(torch.randn(1,512),torch.randn(1,512)).shape

torch.Size([1, 512])

# Transformers from Scratch Pytorch

### Positional Encodings
*   Mathematical
*   Learned Postional Encodings




In [173]:
import torch
import torch.nn as nn

In [174]:
import torch
torch.manual_seed(0)

In [175]:
class positional_embeddings(nn.Module):
  def __init__(self,max_seq_length,embedding_dimension):
    super().__init__()
    self.max_seq_length =max_seq_length
    self.d = embedding_dimension
    self.pos = torch.arange(0,self.max_seq_length,out = torch.empty((1,self.max_seq_length,1)))
    self.i = torch.arange(0,self.d//2,out = torch.empty((1,1,self.d//2)))
    self.pos_embedding = torch.empty(1,self.max_seq_length,self.d)
    self.pos_embedding[:,:,0::2] = torch.sin(self.pos/(torch.pow(10000,(2*self.i/self.d))))
    self.pos_embedding[:,:,1::2] = torch.cos(self.pos/(torch.pow(10000,(2*self.i/self.d))))

  def forward(self,x):
    return x+self.pos_embedding

In [176]:
positional_embeddings(512,768)(torch.randn(16,512,768)).shape

torch.Size([16, 512, 768])

### Embedding Layer

In [177]:
class Embedding_layer(nn.Module):
  def __init__(self,vocab_size,embedding_dimension):
    super().__init__()
    self.embedding_dimension  = embedding_dimension
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(self.vocab_size,self.embedding_dimension)

  def forward(self,x):
    return self.embedding(x)

In [178]:
Embedding_layer(30522,768)(torch.ones(16,512,dtype= int)).shape,positional_embeddings(512,768)(Embedding_layer(30522,768)(torch.ones(16,512,dtype= int))).shape

(torch.Size([16, 512, 768]), torch.Size([16, 512, 768]))

### Padding Mask

In [239]:
class Padding_mask(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,x):
    return x!=0


In [ ]:
Padding_mask()(torch.cat((torch.zeros(16,256,dtype = int),torch.ones(16,256,dtype = int)),dim=1))

In [181]:
(Padding_mask()(torch.cat((torch.zeros(16,256,dtype = int),torch.ones(16,256,dtype = int)),dim=1)) *positional_embeddings(512,768)(Embedding_layer(30522,768)(torch.ones(16,512,dtype= int)))).shape

torch.Size([16, 512, 768])

### Lookahead Mask

In [304]:
class Lookahead_Mask(nn.Module):
  def __init__(self,max_seq_length):
    super().__init__()
    self.max_seq_length = max_seq_length
  def forward(self):
    return torch.tril(torch.ones(self.max_seq_length,self.max_seq_length))

In [309]:
Lookahead_Mask(5)()

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [184]:
Padding_mask()(torch.cat((torch.zeros(16,10,dtype = int),torch.ones(16,10,dtype = int)),dim=1)).shape

torch.Size([16, 20, 1])

In [185]:
res = nn.MultiheadAttention(embed_dim = 10 , num_heads = 2,batch_first =True)(torch.ones(16,20,10),torch.ones(16,20,10),torch.ones(16,20,10),need_weights=False)

In [186]:
res[0].shape

torch.Size([16, 20, 10])

In [187]:
class MHA(nn.Module):
  def __init__(self,embedding_dimension,num_heads):
    super().__init__()
    self.embedding_dimension = embedding_dimension
    self.num_heads = num_heads
    self.attn_dim = self.embedding_dimension//self.num_heads
    self.q = nn.Linear(self.embedding_dimension,self.attn_dim)
    self.k = nn.Linear(self.embedding_dimension,self.attn_dim)
    self.v = nn.Linear(self.embedding_dimension,self.attn_dim)
  def forward(self,x):
    q = self.q(x)
    k = self.k(x)
    v = self.v(x)
    return (q@k.T)/(self.attn_dim**0.5)@v

### Encoder Layer

In [300]:
class Encoder(nn.Module):
  def __init__(self,embedding_dim,num_heads,num_dense,dropout_rate = 0.1):
    super().__init__()
    self.embedding_dim = embedding_dim
    self.num_heads = num_heads
    self.num_dense = num_dense
    self.self_attention = nn.MultiheadAttention(embed_dim = self.embedding_dim , num_heads = self.num_heads,batch_first=True,dropout=dropout_rate)
    self.Dense = nn.Sequential(nn.Linear(self.embedding_dim,num_dense,bias=True),
                               nn.Dropout(0.1 ,inplace=False),
                               nn.Linear(num_dense,self.embedding_dim,bias=True))
    self.norm1 = nn.LayerNorm(self.embedding_dim)
    self.norm2 = nn.LayerNorm(self.embedding_dim)
    # self.dropout1 = nn.Dropout(0.1,inplace=False)
    # self.dropout2 = nn.Dropout(0.1,inplace=False)
  def forward(self,x,mask):
    ### x  -->batchsize,max_seq_len,emb_dim
    # inp = x
    # x = self.self_attention(x,x,x,need_weights=False,key_padding_mask=mask)[0]
    # x = self.norm1(x)
    # x = self.dropout1(x)
    # inp = x+inp
    # x = x+self.Dense(inp)
    # x = self.norm2(x)
    # x = self.dropout2(x)
    # x = x+inp
    # return x
    inp = x
    x = self.self_attention(x,x,x,need_weights=False,key_padding_mask=mask)[0]
    x = x+inp
    inp = self.norm1(x)
    x = x+self.Dense(inp)
    x = x+inp
    x = self.norm2(x)
    return x



In [253]:
Padding_mask()(torch.cat((torch.zeros(1,2,dtype = int),torch.ones(1,3,dtype = int)),dim=1)).shape

torch.Size([1, 5])

In [278]:
Encoder(3,1,5).train()

Encoder(
  (self_attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=3, out_features=3, bias=True)
  )
  (Dense): Sequential(
    (0): Linear(in_features=3, out_features=5, bias=True)
    (1): Linear(in_features=5, out_features=3, bias=True)
  )
  (norm1): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((3,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)

In [299]:
Encoder(3,1,5)(torch.ones(1,5,3),mask =Padding_mask()(torch.cat((torch.zeros(1,2,dtype = int),torch.ones(1,3,dtype = int)),dim=1)))

tensor([[[ 1.3591, -0.3408, -1.0182],
         [ 1.0829,  0.2463, -1.3292],
         [ 1.0159,  0.3441, -1.3600],
         [ 1.3680, -0.3735, -0.9945],
         [ 1.0829,  0.2463, -1.3292]]], grad_fn=<NativeLayerNormBackward0>)

### Decoder Layer

In [351]:
class Decoder(nn.Module):
  def __init__(self,embedding_dim,num_heads,num_dense,max_seq_len,dropout_rate = 0.1):
    super().__init__()
    self.embedding_dim = embedding_dim
    self.num_heads = num_heads
    self.num_dense = num_dense
    self.look_ahead_mask = Lookahead_Mask(max_seq_len)
    self.masked_mha = nn.MultiheadAttention(embed_dim = self.embedding_dim , num_heads = self.num_heads,batch_first=True,dropout=dropout_rate)
    self.mha = nn.MultiheadAttention(embed_dim = self.embedding_dim , num_heads = self.num_heads,batch_first=True,dropout = dropout_rate)
    self.Dense = nn.Sequential(nn.Linear(self.embedding_dim,num_dense,bias=True),
                               nn.Dropout(0.1 ,inplace=False),
                               nn.Linear(num_dense,self.embedding_dim,bias=True))
    self.dropout1 = nn.Dropout(dropout_rate,inplace=False)
    self.norm1 = nn.LayerNorm(self.embedding_dim)
    self.norm2 = nn.LayerNorm(self.embedding_dim)
    self.norm3 = nn.LayerNorm(self.embedding_dim)

  def forward(self,x,enc_mask,enc_inp):
    inp = x
    x = self.masked_mha(x,x,x,need_weights=False,attn_mask = self.look_ahead_mask())[0] ## should i consider padding of target as well
    x = x+inp
    x = self.norm1(x)
    inp = x

    x = self.mha(x,enc_inp,enc_inp,need_weights=False,key_padding_mask=enc_mask)[0]
    x = x+inp
    x = self.norm2(x)
    inp = x

    x = self.Dense(x)
    x = x+inp
    x = self.norm3(x)
    return x


In [352]:
Decoder(10,2,50,6)(torch.ones(1,6,10),Padding_mask()(torch.cat((torch.zeros(1,2,dtype = int),torch.ones(1,4,dtype = int)),dim=1)),torch.ones(1,6,10)).shape

torch.Size([1, 6, 10])

## Transformer Block

In [363]:
class Transformer(nn.Module):
  def __init__(self,vocab_size,embedding_dimension,max_seq_length,num_heads,num_encoder_layers,num_decoder_layers,num_dense,dropout_rate = 0.1):
    super().__init__()
    self.embedding_dimension = embedding_dimension
    self.max_seq_length = max_seq_length
    self.num_heads = num_heads
    self.emb = Embedding_layer(vocab_size,embedding_dimension)
    self.pos = positional_embeddings(max_seq_length,embedding_dimension)
    self.padding_mask = Padding_mask()
    self.Encoder_layer = nn.ModuleList([Encoder(embedding_dimension,num_heads,num_dense,dropout_rate) for _ in range(num_encoder_layers)])
    self.Decoder_layer = nn.ModuleList([Decoder(embedding_dimension,num_heads,num_dense,max_seq_length,dropout_rate) for _ in range(num_decoder_layers)])
  def forward(self,inp_seq,target_seq):
    ## Encoder
    mask = self.padding_mask(inp_seq)   ### batchsize,max_seq_len
    x = self.emb(inp_seq)    ### batchsize,max_seq_len,emb_dim
    x = self.pos(x)    ### batchsize,max_seq_len,emb_dim
    for i in self.Encoder_layer:
      x = i(x,mask)          ### batchsize,max_seq_len,emb_dim
    ## Decoder
    # mask = self.padding_mask(target_seq)
    target_seq = self.emb(target_seq)
    target_seq = self.pos(target_seq)
    for i in self.Decoder_layer:
      x = i(target_seq,mask,x)
    return x


In [365]:
tr(torch.ones(16,20,dtype = int),torch.ones(16,20,dtype=int)).shape

torch.Size([16, 20, 10])

In [364]:
tr = Transformer(vocab_size=100,embedding_dimension=10,max_seq_length=20,num_heads=2,num_encoder_layers=1,num_decoder_layers=1,num_dense=50)
tr

Transformer(
  (emb): Embedding_layer(
    (embedding): Embedding(100, 10)
  )
  (pos): positional_embeddings()
  (padding_mask): Padding_mask()
  (Encoder_layer): ModuleList(
    (0): Encoder(
      (self_attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=10, out_features=10, bias=True)
      )
      (Dense): Sequential(
        (0): Linear(in_features=10, out_features=50, bias=True)
        (1): Dropout(p=0.1, inplace=False)
        (2): Linear(in_features=50, out_features=10, bias=True)
      )
      (norm1): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
    )
  )
  (Decoder_layer): ModuleList(
    (0): Decoder(
      (look_ahead_mask): Lookahead_Mask()
      (masked_mha): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=10, out_features=10, bias=True)
      )
      (mha): MultiheadAttention(
        (out_proj): NonDynamicallyQuan

In [367]:
nn.Transformer(10,2,1,1,50,0.1)(torch.ones(16,20,10),torch.ones(16,20,10)).shape

torch.Size([16, 20, 10])

In [372]:
total = 0
for i,j in Transformer(vocab_size=100,embedding_dimension=10,max_seq_length=20,num_heads=2,num_encoder_layers=1,num_decoder_layers=1,num_dense=50).named_parameters():
  print(i,j.numel())
  total+=j.numel()
print('\n\n\n',total)

emb.embedding.weight 1000
Encoder_layer.0.self_attention.in_proj_weight 300
Encoder_layer.0.self_attention.in_proj_bias 30
Encoder_layer.0.self_attention.out_proj.weight 100
Encoder_layer.0.self_attention.out_proj.bias 10
Encoder_layer.0.Dense.0.weight 500
Encoder_layer.0.Dense.0.bias 50
Encoder_layer.0.Dense.2.weight 500
Encoder_layer.0.Dense.2.bias 10
Encoder_layer.0.norm1.weight 10
Encoder_layer.0.norm1.bias 10
Encoder_layer.0.norm2.weight 10
Encoder_layer.0.norm2.bias 10
Decoder_layer.0.masked_mha.in_proj_weight 300
Decoder_layer.0.masked_mha.in_proj_bias 30
Decoder_layer.0.masked_mha.out_proj.weight 100
Decoder_layer.0.masked_mha.out_proj.bias 10
Decoder_layer.0.mha.in_proj_weight 300
Decoder_layer.0.mha.in_proj_bias 30
Decoder_layer.0.mha.out_proj.weight 100
Decoder_layer.0.mha.out_proj.bias 10
Decoder_layer.0.Dense.0.weight 500
Decoder_layer.0.Dense.0.bias 50
Decoder_layer.0.Dense.2.weight 500
Decoder_layer.0.Dense.2.bias 10
Decoder_layer.0.norm1.weight 10
Decoder_layer.0.norm1.

In [370]:
total = 0
for i,j in nn.Transformer(10,2,1,1,50,0.1).named_parameters():
  print(i,j.numel())
  total+=j.numel()
print('\n\n\n',total)

encoder.layers.0.self_attn.in_proj_weight 300
encoder.layers.0.self_attn.in_proj_bias 30
encoder.layers.0.self_attn.out_proj.weight 100
encoder.layers.0.self_attn.out_proj.bias 10
encoder.layers.0.linear1.weight 500
encoder.layers.0.linear1.bias 50
encoder.layers.0.linear2.weight 500
encoder.layers.0.linear2.bias 10
encoder.layers.0.norm1.weight 10
encoder.layers.0.norm1.bias 10
encoder.layers.0.norm2.weight 10
encoder.layers.0.norm2.bias 10
encoder.norm.weight 10
encoder.norm.bias 10
decoder.layers.0.self_attn.in_proj_weight 300
decoder.layers.0.self_attn.in_proj_bias 30
decoder.layers.0.self_attn.out_proj.weight 100
decoder.layers.0.self_attn.out_proj.bias 10
decoder.layers.0.multihead_attn.in_proj_weight 300
decoder.layers.0.multihead_attn.in_proj_bias 30
decoder.layers.0.multihead_attn.out_proj.weight 100
decoder.layers.0.multihead_attn.out_proj.bias 10
decoder.layers.0.linear1.weight 500
decoder.layers.0.linear1.bias 50
decoder.layers.0.linear2.weight 500
decoder.layers.0.linear2.

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [211]:
nn.TransformerEncoderLayer(d_model=10,nhead=2,batch_first=True,dim_feedforward=50)

TransformerEncoderLayer(
  (self_attn): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=10, out_features=10, bias=True)
  )
  (linear1): Linear(in_features=10, out_features=50, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (linear2): Linear(in_features=50, out_features=10, bias=True)
  (norm1): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
  (norm2): LayerNorm((10,), eps=1e-05, elementwise_affine=True)
  (dropout1): Dropout(p=0.1, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
)

# Transformers Cumulatively in pyTorch

In [374]:
class positional_embeddings(nn.Module):
  def __init__(self,max_seq_length,
               embedding_dimension):
    super().__init__()
    self.max_seq_length =max_seq_length
    self.d = embedding_dimension
    self.pos = torch.arange(0,self.max_seq_length,out = torch.empty((1,self.max_seq_length,1)))
    self.i = torch.arange(0,self.d//2,out = torch.empty((1,1,self.d//2)))
    self.pos_embedding = torch.empty(1,self.max_seq_length,self.d)
    self.pos_embedding[:,:,0::2] = torch.sin(self.pos/(torch.pow(10000,(2*self.i/self.d))))
    self.pos_embedding[:,:,1::2] = torch.cos(self.pos/(torch.pow(10000,(2*self.i/self.d))))

  def forward(self,x):
    return x+self.pos_embedding



class Embedding_layer(nn.Module):
  def __init__(self,vocab_size,
               embedding_dimension):
    super().__init__()
    self.embedding_dimension  = embedding_dimension
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(self.vocab_size,self.embedding_dimension)

  def forward(self,x):
    return self.embedding(x)



class Padding_mask(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,x):
    return x!=0

class Lookahead_Mask(nn.Module):
  def __init__(self,max_seq_length):
    super().__init__()
    self.max_seq_length = max_seq_length
  def forward(self):
    return torch.tril(torch.ones(self.max_seq_length,self.max_seq_length))

class Encoder(nn.Module):
  def __init__(self,embedding_dim,
               num_heads,
               num_dense,
               dropout_rate = 0.1):
    super().__init__()
    self.embedding_dim = embedding_dim
    self.num_heads = num_heads
    self.num_dense = num_dense
    self.self_attention = nn.MultiheadAttention(embed_dim = self.embedding_dim , num_heads = self.num_heads,batch_first=True,dropout=dropout_rate)
    self.Dense = nn.Sequential(nn.Linear(self.embedding_dim,num_dense,bias=True),
                               nn.Dropout(0.1 ,inplace=False),
                               nn.Linear(num_dense,self.embedding_dim,bias=True))
    self.norm1 = nn.LayerNorm(self.embedding_dim)
    self.norm2 = nn.LayerNorm(self.embedding_dim)
  def forward(self,x,mask):
    ### x  -->batchsize,max_seq_len,emb_dim

    inp = x
    x = self.self_attention(x,x,x,need_weights=False,key_padding_mask=mask)[0]
    x = x+inp
    inp = self.norm1(x)
    x = x+self.Dense(inp)
    x = x+inp
    x = self.norm2(x)
    return x


class Decoder(nn.Module):
  def __init__(self,embedding_dim,
               num_heads,
               num_dense,
               max_seq_len,
               dropout_rate = 0.1):
    super().__init__()
    self.embedding_dim = embedding_dim
    self.num_heads = num_heads
    self.num_dense = num_dense
    self.look_ahead_mask = Lookahead_Mask(max_seq_len)
    self.masked_mha = nn.MultiheadAttention(embed_dim = self.embedding_dim , num_heads = self.num_heads,batch_first=True,dropout=dropout_rate)
    self.mha = nn.MultiheadAttention(embed_dim = self.embedding_dim , num_heads = self.num_heads,batch_first=True,dropout = dropout_rate)
    self.Dense = nn.Sequential(nn.Linear(self.embedding_dim,num_dense,bias=True),
                               nn.Dropout(0.1 ,inplace=False),
                               nn.Linear(num_dense,self.embedding_dim,bias=True))
    self.dropout1 = nn.Dropout(dropout_rate,inplace=False)
    self.norm1 = nn.LayerNorm(self.embedding_dim)
    self.norm2 = nn.LayerNorm(self.embedding_dim)
    self.norm3 = nn.LayerNorm(self.embedding_dim)

  def forward(self,x,
              enc_mask,
              enc_inp):
    inp = x
    x = self.masked_mha(x,x,x,need_weights=False,attn_mask = self.look_ahead_mask())[0] ## should i consider padding of target as well
    x = x+inp
    x = self.norm1(x)
    inp = x

    x = self.mha(x,enc_inp,enc_inp,need_weights=False,key_padding_mask=enc_mask)[0]
    x = x+inp
    x = self.norm2(x)
    inp = x

    x = self.Dense(x)
    x = x+inp
    x = self.norm3(x)
    return x

class Transformer(nn.Module):
  def __init__(self,src_vocab_size,
               tar_vocab_size,
               embedding_dimension,
               max_seq_length,
               num_heads,
               num_encoder_layers,
               num_decoder_layers,
               num_dense,
               dropout_rate = 0.1):
    super().__init__()
    self.embedding_dimension = embedding_dimension
    self.max_seq_length = max_seq_length
    self.num_heads = num_heads
    self.emb_src = Embedding_layer(src_vocab_size,embedding_dimension)
    self.emb_target = Embedding_layer(tar_vocab_size,embedding_dimension)
    self.pos = positional_embeddings(max_seq_length,embedding_dimension)
    self.padding_mask = Padding_mask()
    self.Encoder_layer = nn.ModuleList([Encoder(embedding_dimension,num_heads,num_dense,dropout_rate) for _ in range(num_encoder_layers)])
    self.Decoder_layer = nn.ModuleList([Decoder(embedding_dimension,num_heads,num_dense,max_seq_length,dropout_rate) for _ in range(num_decoder_layers)])
  def forward(self,inp_seq,target_seq):
    ## Encoder
    mask = self.padding_mask(inp_seq)   ### batchsize,max_seq_len
    x = self.emb_src(inp_seq)    ### batchsize,max_seq_len,emb_dim
    x = self.pos(x)    ### batchsize,max_seq_len,emb_dim
    for i in self.Encoder_layer:
      x = i(x,mask)          ### batchsize,max_seq_len,emb_dim
    ## Decoder
    # mask = self.padding_mask(target_seq)
    target_seq = self.emb_target(target_seq)
    target_seq = self.pos(target_seq)
    for i in self.Decoder_layer:
      x = i(target_seq,mask,x)
    return x


In [375]:
tr = Transformer(src_vocab_size=30522,tar_vocab_size=30522,embedding_dimension=768,max_seq_length=512,num_heads=8,num_encoder_layers=6,num_decoder_layers=6,num_dense=2048,dropout_rate = 0.1)
tr

Transformer(
  (emb_src): Embedding_layer(
    (embedding): Embedding(30522, 768)
  )
  (emb_target): Embedding_layer(
    (embedding): Embedding(30522, 768)
  )
  (pos): positional_embeddings()
  (padding_mask): Padding_mask()
  (Encoder_layer): ModuleList(
    (0-5): 6 x Encoder(
      (self_attention): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
      )
      (Dense): Sequential(
        (0): Linear(in_features=768, out_features=2048, bias=True)
        (1): Dropout(p=0.1, inplace=False)
        (2): Linear(in_features=2048, out_features=768, bias=True)
      )
      (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    )
  )
  (Decoder_layer): ModuleList(
    (0-5): 6 x Decoder(
      (look_ahead_mask): Lookahead_Mask()
      (masked_mha): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=768, o

In [379]:
def count_parameters(model):
    total = 0
    trainable =0
    for name, parameter in model.named_parameters():
        param = parameter.numel()
        total+=param
        if parameter.requires_grad:
          trainable+=param
    return f"Total_parameters: {total}, trainable_parameters: {trainable}, fraction of trainable parameters: {(trainable/total)*100}%"
count_parameters(tr)

'Total_parameters: 127233024, trainable_parameters: 127233024, fraction of trainable parameters: 100.0%'

# sOME

In [ ]:
x = torch.cat((torch.ones(16,256,dtype=int),torch.zeros(16,256,dtype=int)),dim=1)
mask = Padding_mask()(x)


In [ ]:
x = emb(x)
x = pos(x)

In [ ]:
(x*mask)[0]

tensor([[-0.4304,  1.0703, -0.2786,  ...,  3.2687, -0.2441,  0.3044],
        [ 0.4111,  0.6106,  0.5498,  ...,  3.2687, -0.2440,  0.3044],
        [ 0.4789, -0.3459,  0.6494,  ...,  3.2687, -0.2439,  0.3044],
        ...,
        [-0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<SelectBackward0>)

In [ ]:
vocab_size = 30522
embedding_dimension = 768
max_seq_length = 512
emb = Embedding_layer(vocab_size,embedding_dimension)
pos = positional_embeddings(max_seq_length,embedding_dimension)

In [ ]:
x = torch.ones(16,512,dtype = torch.long)
x = emb(x)
x = pos(x)

In [ ]:
x.shape

torch.Size([16, 512, 768])

In [ ]:
a = torch.nn.MultiheadAttention(1024,4)

In [ ]:
torch.ones(3,10)

tensor([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [ ]:
a(torch.ones(3,10),torch.ones(3,10),torch.ones(3,10))[0].shape

torch.Size([3, 10])

In [ ]:
for i in a.parameters():
  print(i.shape)

torch.Size([30, 10])
torch.Size([30])
torch.Size([10, 10])
torch.Size([10])


In [ ]:
class Attention(nn.Module):
  def __init__(self,input_dimension, attention_dimension):
    super().__init__()
    self.attention_dimension = attention_dimension
    self.input_dimension = input_dimension
    self.q = nn.Linear(input_dimension,attention_dimension) #nn.Parameter(torch.randn(input_dimension,attention_dimension))
    self.k =  nn.Linear(input_dimension,attention_dimension) #nn.Parameter(torch.randn(input_dimension,attention_dimension))
    self.v =  nn.Linear(input_dimension,attention_dimension) #nn.Parameter(torch.randn(input_dimension,attention_dimension))
    # self.o = nn.Parameter(torch.randn(input_dimension,attention_dimension))
    self.softmax = nn.Softmax(dim=-1)

  def forward(self,x):
    query = self.q(x)
    key = self.k(x)
    value = self.v(x)
    return (self.softmax((query@key.T)/(self.attention_dimension**0.5)))@value


In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self,input_dimension,num_heads):
    super().__init__()
    assert input_dimension%num_heads==0
    attention_dimension = input_dimension//num_heads
    self.num_heads = num_heads
    self.Attention = nn.ModuleList([Attention(input_dimension,attention_dimension) for _ in range(num_heads)])
    self.o = nn.Linear(num_heads*attention_dimension,input_dimension)

  def forward(self,x):
    return self.o(torch.cat([a(x) for a in self.Attention],dim=-1))

In [ ]:
multihead_attention = MultiHeadAttention(768,12)

In [ ]:
multihead_attention(torch.ones(3,10)).shape

torch.Size([3, 10])

In [ ]:
total =0
for i in multihead_attention.parameters():
  total+=i.numel()
total

2362368

In [ ]:
total =0
for i in a.parameters():
  total+=i.numel()
total

2362368

In [ ]:
input_dimension  = 16
attention_dimension = 4

In [ ]:
x = torch.randn(5,input_dimension)

In [ ]:
a = Attention(input_dimension,attention_dimension)


In [ ]:
a(x)

tensor([[ 2.7866, -0.4228,  7.5092,  4.9380],
        [ 2.1012,  1.3627,  0.8742, -0.4853],
        [-5.4467, -6.8884,  1.8050, -1.9524],
        [ 2.7875, -0.4211,  7.5061,  4.9354],
        [-5.4916, -6.9169,  1.7893, -1.9728]], grad_fn=<MmBackward0>)

In [ ]:
query = x@a.q
key = x@a.k
value = x@a.v
query.shape,key.shape,value.shape

(torch.Size([5, 4]), torch.Size([5, 4]), torch.Size([5, 4]))

In [ ]:
(a.softmax((query@key.T)/(attention_dimension**0.5)))@value

tensor([[ 1.1145,  0.1980,  0.9490, -4.8318],
        [ 2.7070, -1.6848,  1.0841,  1.3753],
        [-1.6475,  2.5051, -0.5788, -0.2157],
        [ 3.3452, -2.4296,  1.1502,  3.7107],
        [-1.6473,  2.5049, -0.5787, -0.2156]], grad_fn=<MmBackward0>)

In [ ]:
loss_fn  = nn.MSELoss()

In [ ]:
a = torch.randn(10)
b = torch.randn(10)
loss_fn(a,b)

tensor(2.1294)

In [ ]:
mask = torch.ones(10)
mask[:5] =torch.zeros(5)

In [ ]:
mask[5:]= torch.ones(5)*-100

In [ ]:
mask

tensor([   0.,    0.,    0.,    0.,    0., -100., -100., -100., -100., -100.])

In [ ]:
def custom_loss_fn(a,b,mask=None):
  if mask is None:
    return loss_fn(a,b)
  zeros = torch.zeros_like(mask)
  zeros[torch.where(mask==0)] = torch.ones(torch.where(mask==0)[0].shape)
  return loss_fn(a*zeros,b*zeros)

In [ ]:
custom_loss_fn(a,b,mask)

tensor(0.7751)

In [ ]:
loss_fn(a,b)

tensor(2.1294)

In [ ]:
a,b

(tensor([ 0.1040,  0.7778, -1.0505, -1.5696,  0.3505, -0.2407, -0.5537, -0.2028,
          0.9568,  0.1639]),
 tensor([-0.2057,  0.1776, -0.4010,  0.9425, -0.3997,  0.6875,  1.5685,  0.8278,
         -1.7007, -0.0678]))

In [ ]:
transformer = torch.nn.Transformer()

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
data = torch.randn(1000,512)
data_y = torch.randn(1000,512)

In [ ]:
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(transformer.parameters(),lr=5e-5)

In [ ]:
for epoch in range(5):
  output = transformer(data,data_y)
  l = loss_fn(output,data_y)
  l.backward()
  optimizer.step()
  print(l)

tensor(1.9532, grad_fn=<MseLossBackward0>)


KeyboardInterrupt: 

In [ ]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
#load english to french dataset
dataset = load_dataset("opus_books", "en-fr")

README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 127085
    })
})

In [ ]:
def tokenize(example):
  example['input_ids'] =  tokenizer(example['translation']['en'],truncation = True,padding = "max_length")['input_ids']
  example['label'] = tokenizer(example['translation']['fr'],truncation = True,padding = 'max_length')['input_ids']
  return example

In [ ]:
ds = dataset['train'].take(20000).map(tokenize)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [ ]:
ds

Dataset({
    features: ['id', 'translation', 'input_ids', 'label'],
    num_rows: 20000
})

In [ ]:
ds[0]

In [ ]:
tokenizer.decode(ds[40]['input_ids']),tokenizer.decode(ds[40]['label'])

('[CLS] her face was thin and refined, but worn with anxiety, and, at sight of her, i do not know what misgiving made me stand still on the first step, in front of the gate. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(transformer.parameters(),lr=5e-3)

In [ ]:
mini_batch_size = 16
for epoch in range(5):
  for mini_batch_index in range(len(ds)//mini_batch_size):
    mini_batch = ds[mini_batch_index*mini_batch_size:(mini_batch_index+1)*mini_batch_size]
    # Change 1: Convert input data to float32
    data = torch.tensor(mini_batch['input_ids'], dtype=torch.float32)
    # Change 2: Convert target data to float32 if loss function requires it
    data_y = torch.tensor(mini_batch['label'], dtype=torch.float32)
    output = transformer(data,data_y)
    l = loss_fn(output,data_y)
    l.backward()
    optimizer.step()
    print(f"loss at epoch {epoch} and step {mini_batch_index} :{l}")

loss at epoch 0 and step 0 :9168400.0
loss at epoch 0 and step 1 :12563897.0
loss at epoch 0 and step 2 :11738259.0
loss at epoch 0 and step 3 :11250613.0
loss at epoch 0 and step 4 :7700791.5
loss at epoch 0 and step 5 :9695593.0
loss at epoch 0 and step 6 :13367944.0
loss at epoch 0 and step 7 :7968085.0
loss at epoch 0 and step 8 :10144013.0
loss at epoch 0 and step 9 :4948914.5
loss at epoch 0 and step 10 :9211023.0
loss at epoch 0 and step 11 :6166519.5
loss at epoch 0 and step 12 :5968641.5


KeyboardInterrupt: 

In [ ]:
data_y[15]

# Using transformer module of torch

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('google-t5/t5-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:81: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import torch.nn as nn
import torch

In [ ]:
class Transformer_en_fr(nn.Module):
  def __init__(self,en_vocab_size=30522,embedding_dim=512,fr_vocab_size=30522,nhead = 8):
    super().__init__()
    self.en_embedding = nn.Embedding(en_vocab_size,embedding_dim)
    self.fr_embedding = nn.Embedding(fr_vocab_size,embedding_dim)
    self.transformer = torch.nn.Transformer(d_model=embedding_dim,nhead = nhead,batch_first = True )
    self.linear = nn.Linear(embedding_dim,fr_vocab_size)

  def forward(self,x,y):
    x = self.en_embedding(x)
    y = self.fr_embedding(y)
    output = self.transformer(x,y)
    return self.linear(output)

In [ ]:
transformer_en_fr = Transformer_en_fr()
# transformer_en_fr

In [ ]:
30522
16,300
16,300,512     16,300,512   -->inputs
16, 300,512    -->output from transformer
16,300,30522   -->output from linear
16,300,1 --> after cross entropy


In [ ]:
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
dataset  = load_dataset("opus_books","en-fr")

In [ ]:
tokenizer.vocab_size

32100

In [ ]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.vocab_size


32100

In [ ]:
tokenizer.add_special_tokens({'bos_token':'<s>'})
tokenizer.vocab_size


32100

In [ ]:
def tokenize(example):
  example['en_input'] =  tokenizer(example['translation']['en'],truncation = True,padding='max_length',max_length= 512)['input_ids']
  fr_tokens = tokenizer(example['translation']['fr'],truncation = True,padding='max_length',max_length = 513)['input_ids']
  example['fr_input'] = fr_tokens[:-1]
  example['label'] = fr_tokens[1:]
  # example['label'] = tokenizer(example['translation']['fr'][1:],truncation = True,padding = 'max_length',return_tensors = 'pt')['input_ids']
  return example

In [ ]:
ds = dataset['train'].take(5000).map(tokenize)

In [ ]:
ds = ds.remove_columns(['id', 'translation'])

In [ ]:
ds

Dataset({
    features: ['en_input', 'fr_input', 'label'],
    num_rows: 5000
})

In [ ]:
optimizer = torch.optim.Adam(transformer_en_fr.parameters(),lr=5e-3)
loss_function = torch.nn.CrossEntropyLoss(reduction='none',ignore_index=0)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
ds.set_format(type='torch')

In [ ]:
from torch.utils.data import DataLoader
dl = DataLoader(ds,batch_size = 8)

In [ ]:
transformer_en_fr.to(device)

Transformer_en_fr(
  (en_embedding): Embedding(30522, 512)
  (fr_embedding): Embedding(30522, 512)
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
          )
          (linear1): Linear(in_features=512, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=512, bias=True)
          (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
 

In [ ]:
epochs = 10
step = 0
for epoch in range(epochs):
  for i in dl:
    x,y,label = i['en_input'],i['fr_input'],i['label']
    x=x.to(device)
    y=y.to(device)
    label=label.to(device)
    optimizer.zero_grad()
    res = transformer_en_fr(x,y)
    loss = loss_function(res.permute(0,2,1),label).mean(dim=1)
    # Create a mask for non-zero values
    mask = loss != 0

    # Calculate the mean of non-zero values
    loss = loss[mask].mean()
    loss.backward()
    optimizer.step()
    if step%100==0:
      print("loss",loss)
      print("Translation from ")
      text = "<s>"
      print("hey how are you")
      for i in range(10):
        text+=tokenizer.decode(torch.argmax(transformer_en_fr(tokenizer("hey how are you",return_tensors='pt')['input_ids'][0].to(device),tokenizer(text,return_tensors='pt')['input_ids'][0].to(device))[-1]),skip_special_tokens=True)
      print('\033[92m'+text +'\033[0m','\n\n')
    step+=1

loss tensor(0.2534, device='cuda:0', grad_fn=<MeanBackward0>)
Translation from 
hey how are you
<s> 




RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling cublasLtMatmul with transpose_mat1 1 transpose_mat2 0 m 2048 n 4096 k 512 mat1_ld 512 mat2_ld 512 result_ld 2048 abcType 0 computeType 68 scaleType 0

In [ ]:
print("x shape:", x.shape, "y shape:", y.shape, "label shape:", label.shape)


x shape: torch.Size([8, 512]) y shape: torch.Size([8, 512]) label shape: torch.Size([8, 512])


# EN_FR